In [1]:
from ever_crisis_gacha_simulator.classes.gacha_sim import GachaSim
from ever_crisis_gacha_simulator.banner_info_and_stamp_cards import ZACK_FF9_CROSSOVER_BANNER
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use("cyberpunk")
plt.rcParams.update({"figure.dpi": 600})

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [7]:
def ob_analysis_wrapper(criterion_overboost_value):
    """
    A wrapper around GachaSim instantiation for our the FF9 crossover banner, targeting the Beatrix Sword.
    """

    gs = GachaSim(
        session_criterion="overboost",
        criterion_value=criterion_overboost_value,
        target_weapon_type="featured",
        banner_info=ZACK_FF9_CROSSOVER_BANNER,
        seed_value=1337,
        starting_weapon_parts=0,
        num_simulations=500_000,
    )

    return gs

# A dictionary for only one input is a bit overkill, so I'll be skipping that step and just inputting
# the overboost value. 

In [ ]:
def viz_wrapper(
    gs, 
    column,
    values=None,
    probs=None,
    ):

    if probs and values:
        print("ERROR: Values passed for both `probs` and `values`. Only pass values for one.")
        return
    
    ORB_HEX_CODES = ["#0093FF", "#CB00FF", "#FFC800"]
    MAX_X_TICKS = 15
    FONT_SIZE = 12
    
    if probs:
        colors = sns.blend_palette(colors=ORB_HEX_CODES, n_colors=len(probs))
        colors.reverse()
    elif values:
        colors = sns.blend_palette(colors=ORB_HEX_CODES, n_colors=len(values))
        colors.reverse()
    else:
        colors = None
    
    plot = gs.visualize_results(outcome=column)

    x_max = plot.ax.get_xlim()[1]
    
    if column == "num_crystals_spent":
        crystal_count_by = 3_000

        # Ensure adequate tick mark spacing for x-axis
        while x_max / crystal_count_by > MAX_X_TICKS:
            crystal_count_by += 3_000
        
        plot.ax.set_xticks(np.arange(0, x_max+1, crystal_count_by))
        plot.ax.set_xticklabels([f"{label:,.0f}" for label in plot.ax.get_xticks()])
        # plot.set_xticklabels(rotation=60)
    elif column == "targeted_weapon_parts":
        wpn_parts_count_by = 200

        while x_max / wpn_parts_count_by > MAX_X_TICKS:
            wpn_parts_count_by += 200
        
        plot.ax.set_xticks(np.arange(0, x_max+1, wpn_parts_count_by))
        plot.ax.set_xticklabels([f"{label:,.0f}" for label in plot.ax.get_xticks()])
        
    plot.set_xticklabels(rotation=45, fontsize=FONT_SIZE, fontweight='semibold')
    plot.ax.set_yticks(np.arange(0, 101, 10))
    plot.set_yticklabels(fontsize=FONT_SIZE, fontweight='semibold') 

    if probs:
        values = [int(np.percentile(gs.sim_results[column], prob)) for prob in probs]
    
    # Horizontal lines with labels for values of interest
    if values or probs:

        if column == "num_crystals_spent":
            legend_label = "Crystals"
        elif column == "targeted_weapon_parts":
            legend_label = "Overboost"
        else:
            legend_label == "Stamps"

        WEAPON_PARTS_TO_OVERBOOST = 200

        # if column != "targeted_weapon_parts":
        #     colors.reverse()
        
        for index, value in enumerate(sorted(values, reverse=True)):

            if column == "targeted_weapon_parts":
                converted_value = int(value / WEAPON_PARTS_TO_OVERBOOST - 1)
            
            value_prob = gs.return_value_probability(column=column, value=value)
            
            plot.ax.hlines(
                y=value_prob,
                linestyles="dashed",
                linewidth=2,
                xmin=0,
                xmax=x_max,
                color=colors[index],
                alpha=0.75,
                label=f"{value_prob}% ({value:,} {legend_label})" if column != "targeted_weapon_parts" else f"{value_prob}% ({legend_label} {converted_value})",
            )

        plot.ax.legend(
            title="Probability Lines", 
            prop={"weight": "semibold"}, 
            title_fontproperties={"weight": "semibold"}, 
            bbox_to_anchor=(1.60, 0.6)
            )

# Crystals for OB6 of Zack's weapon

In [ ]:
ob6 = ob_analysis_wrapper(6)

ob6.run_sims(n_jobs=-1)

In [ ]:
OUTCOME = "num_crystals_spent"
PROBS = [25, 50, 75, 80, 90, 95]
GS = ob6

viz_wrapper(gs=GS, column=OUTCOME, probs=PROBS)

# Crystals for OB1 of Zack's weapon

In [ ]:
ob1 = ob_analysis_wrapper(1)

ob1.run_sims(n_jobs=-1)

In [ ]:
OUTCOME = "num_crystals_spent"
PROBS = [25, 50, 75, 80, 90, 95]
GS = ob1

viz_wrapper(gs=GS, column=OUTCOME, probs=PROBS)